In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re

def get_sentences_from_url(URL, label):
    html = urlopen(URL).read()
    soup = BeautifulSoup(html, 'html.parser')
    text = soup.get_text().replace('--','-')
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return pd.DataFrame({'text': sentences, 'label': [label] * len(sentences)})

# Загрузка данных трех различных авторов
finger_text = get_sentences_from_url('http://az.lib.ru/f/figner_w_n/text_0090.shtml', 'finger')
koch_text = get_sentences_from_url('http://az.lib.ru/k/koh_m/text_1888_shakespeare-oldorfo.shtml', 'koch')
reiter_text = get_sentences_from_url('http://az.lib.ru/r/rejter_g/text_1897_der_lebenskunstler-oldorfo.shtml', 'reiter')

# Проверка успешности загрузки данных
print(f'Finger data shape: {finger_text.shape}')
print(f'Koch data shape: {koch_text.shape}')
print(f'Reiter data shape: {reiter_text.shape}')

# Объединение данных
data = pd.concat([finger_text, koch_text, reiter_text], ignore_index=True)
data = data.reset_index(drop=True)

# Удаление пустых строк
data = data[data['text'].str.strip().astype(bool)]

# Проверка пустого DataFrame
if data.empty:
    raise ValueError("The combined dataset is empty. Please check the URLs and ensure they return data.")

X = data['text']
y = data['label']

# Разделение данных на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42, stratify=y)

# Векторизация текста с использованием TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=None)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Обучение модели RandomForest
model = RandomForestClassifier(random_state=42)
model.fit(X_train_vec, y_train)

# Оценка модели
y_pred = model.predict(X_test_vec)
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

# Гиперпараметрическая оптимизация с использованием GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
}

grid_search = GridSearchCV(model, param_grid, cv=4, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_vec, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

# Оценка модели с лучшими гиперпараметрами
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test_vec)
print(classification_report(y_test, y_pred_best))
print("Accuracy with best model:", accuracy_score(y_test, y_pred_best))


Tolstoy data shape: (4448, 2)
Goncharov data shape: (6504, 2)
Chekhov data shape: (1412, 2)
              precision    recall  f1-score   support

      finger       0.90      0.88      0.89       890
        koch       0.92      0.85      0.88      1301
      reiter       0.43      0.60      0.50       282

    accuracy                           0.83      2473
   macro avg       0.75      0.78      0.76      2473
weighted avg       0.85      0.83      0.84      2473

Accuracy: 0.8334007278608977
Best parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
Best cross-validation score: 0.8276699029126213
              precision    recall  f1-score   support

      finger       0.93      0.86      0.89       890
        koch       0.79      0.97      0.87      1301
      reiter       0.98      0.15      0.25       282

    accuracy                           0.84      2473
   macro avg       0.90      0.66      0.67      2473
weighted avg       

In [10]:
# Разделение данных на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42, stratify=y)

# Векторизация текста с использованием TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=None)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Обучение модели RandomForest с лучшими гиперпараметрами
best_params = {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
best_model = RandomForestClassifier(**best_params, random_state=42)
best_model.fit(X_train_vec, y_train)

# Оценка модели
y_pred_best = best_model.predict(X_test_vec)
print(classification_report(y_test, y_pred_best))
print("Accuracy with best model:", accuracy_score(y_test, y_pred_best))

# Перекрестная проверка с лучшими гиперпараметрами
cv_results_best = cross_validate(best_model, X_train_vec, y_train, cv=4, return_train_score=True)
print("Cross-validation train scores with best model:", cv_results_best['train_score'])
print("Cross-validation test scores with best model:", cv_results_best['test_score'])
print("Average cross-validation test score with best model:", np.mean(cv_results_best['test_score']))


              precision    recall  f1-score   support

      finger       0.93      0.86      0.89       890
        koch       0.79      0.97      0.87      1301
      reiter       0.98      0.15      0.25       282

    accuracy                           0.84      2473
   macro avg       0.90      0.66      0.67      2473
weighted avg       0.86      0.84      0.81      2473

Accuracy with best model: 0.8378487666801455
Cross-validation train scores with best model: [0.85558252 0.850863   0.85760518 0.85490831]
Cross-validation test scores with best model: [0.82686084 0.83090615 0.82686084 0.82605178]
Average cross-validation test score with best model: 0.8276699029126213


Благодарю за Ваш комментарий: "Код вообще хоть раз запускался? Где результаты?" Ваши острые наблюдения всегда вдохновляют меня на новые свершения.

На всякий случай напоминаю, что код был не только написан, но и успешно запущен. Более того, результаты, которые так волнуют Вас, были представлены в соответствующих блоках вывода. Возможно, их сложно заметить среди всех строк кода и пояснительных комментариев, которые я включила, чтобы сделать проект понятным и прозрачным.

Конечно, я понимаю, что в условиях недостатка времени и желания вникать в детали, такие вещи могут ускользать. Если бы я знала, что мне нужно напомнить о необходимости просмотра конечного вывода, я бы сделал это заранее. Наверное, нужно было крупными буквами в начале работы написать: "ВНИМАНИЕ! РЕЗУЛЬТАТЫ В КОНЦЕ!"

Позвольте мне снова продемонстрировать, что код не только запускался, но и показывал результаты, которые, безусловно, были легко доступны для внимательного глаза:

In [21]:
def classify_sentence(sentence):
    sentence_vec = vectorizer.transform([sentence])
    prediction = best_model.predict(sentence_vec)
    return prediction[0]

new_sentence = "Докторъ Альтгаусъ былъ нѣсколько..."
predicted_author = classify_sentence(new_sentence)
print(predicted_author)

reiter


In [20]:
new_sentence = "Часов через пять пароход..."
predicted_author = classify_sentence(new_sentence)
print(predicted_author)

finger


In [19]:
new_sentence = "Принято за правило, что великіе люди, и въ особенности поэты..."
predicted_author = classify_sentence(new_sentence)
print(predicted_author)

koch
